# Bronze-zu-Silver Agent Flow

## Überblick
Dieses Notebook demonstriert den **Datenbereinigungsprozess** der Agentic ELT Pipeline.
Der Silver Layer transformiert rohe Bronze-Daten in bereinigte, standardisierte Tabellen.

### Agent-Workflow:
1. **Silver Draft Agent** → Analysiert Datenqualität und plant Transformationen
2. **Silver Builder Agent** → Generiert ausführbaren Python-Code
3. **Silver Runner** → Führt Bereinigung aus und erstellt Silver-Tabellen

### Was Sie lernen:
- Wie LLM-Agents Datenqualitätsprobleme automatisch erkennen
- Welche Transformationen für Datenbereinigung angewendet werden
- Wie Agents Code für ETL-Prozesse generieren

## 📋 Setup und Konfiguration

**Was passiert hier:**
- Laden der benötigten Python-Bibliotheken
- Definition der Pfade zu Bronze- und Silver-Artefakten
- Überprüfung der Datenverfügbarkeit

In [1]:
import json
import pandas as pd
from pathlib import Path
import yaml

# Setzen Sie hier Ihre Run-ID
run_id = "20260124_231016_#6470e523"

# Pfade definieren
bronze_dir = Path(f"../artifacts/bronze/{run_id}")
silver_dir = Path(f"../artifacts/silver/{run_id}")

print(f"🔍 Überprüfe Datenverzeichnisse:")
print(f"Bronze Directory: {bronze_dir} ({'✅ vorhanden' if bronze_dir.exists() else '❌ fehlt'})")
print(f"Silver Directory: {silver_dir} ({'✅ vorhanden' if silver_dir.exists() else '❌ fehlt'})")

🔍 Überprüfe Datenverzeichnisse:
Bronze Directory: ..\artifacts\bronze\20260124_231016_#6470e523 (✅ vorhanden)
Silver Directory: ..\artifacts\silver\20260124_231016_#6470e523 (✅ vorhanden)


## 🥉 Bronze Layer Inspektion

**Was passiert hier:**
- Analyse der rohen Eingangsdaten aus dem Bronze Layer
- Überprüfung der Metadaten und Datenstrukturen
- Identifikation der verfügbaren Tabellen und deren Eigenschaften

**Warum wichtig:**
Der Silver Draft Agent benötigt diese Informationen, um Datenqualitätsprobleme zu erkennen und Transformationsstrategien zu entwickeln.

In [2]:
# Bronze Metadata analysieren (robust + validiert)
from pathlib import Path
import yaml

def _load_yaml(path: Path) -> dict:
    if not path.exists():
        raise FileNotFoundError(f"Metadata nicht gefunden: {path.resolve()}")
    data = yaml.safe_load(path.read_text(encoding="utf-8")) or {}
    if not isinstance(data, dict):
        raise TypeError(f"Unerwartetes YAML-Format (kein Mapping) in {path.resolve()}: {type(data)}")
    return data

def _require(meta: dict, *keys: str, path: Path = None):
    cur = meta
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            available = list(cur.keys()) if isinstance(cur, dict) else None
            where = f" in {path.resolve()}" if path else ""
            raise KeyError(
                f"Fehlender Key: {'/'.join(keys)}{where}. "
                f"Verfügbare Keys auf dieser Ebene: {available}"
            )
        cur = cur[k]
    return cur

bronze_metadata_path = bronze_dir / "data" / "metadata.yaml"
bronze_metadata = None

try:
    bronze_metadata = _load_yaml(bronze_metadata_path)

    # Pflichtfelder (früh & laut scheitern, falls Schema nicht passt)
    run_id_val = _require(bronze_metadata, "run", "run_id", path=bronze_metadata_path)
    layer_val  = _require(bronze_metadata, "run", "layer",  path=bronze_metadata_path)

    print("📊 Bronze Layer Metadata:")
    print(f"Run ID: {run_id_val}")
    print(f"Layer: {layer_val}")

    # Optionale Felder
    summary = bronze_metadata.get("summary", {}) or {}
    print(f"\n📈 Ingestion Zusammenfassung:")
    print(f"  Dateien gesamt: {summary.get('files_total')}")
    print(f"  Dateien erfolgreich: {summary.get('files_success')}")
    print(f"  Dateien fehlgeschlagen: {summary.get('files_failed')}")

except Exception as e:
    print("❌ Fehler beim Laden/Validieren der Bronze-Metadata:")
    print(f"  {e}")
    print("\n🔎 Debug:")
    print(f"  Pfad: {bronze_metadata_path}")
    print(f"  Absolut: {bronze_metadata_path.resolve()}")
    print(f"  Existiert: {bronze_metadata_path.exists()}")

    if isinstance(bronze_metadata, dict):
        print(f"  Top-Level Keys: {list(bronze_metadata.keys())}")
        run_block = bronze_metadata.get("run")
        if isinstance(run_block, dict):
            print(f"  run Keys: {list(run_block.keys())}")
        else:
            print(f"  run Typ: {type(run_block)}")


📊 Bronze Layer Metadata:
Run ID: 20260124_231016_#6470e523
Layer: bronze

📈 Ingestion Zusammenfassung:
  Dateien gesamt: 9
  Dateien erfolgreich: 9
  Dateien fehlgeschlagen: 0


In [3]:
# Bronze Tabellen inspizieren
bronze_data_dir = bronze_dir / "data"
csv_files = list(bronze_data_dir.glob("*.csv"))

print(f"🗂️ Bronze Tabellen ({len(csv_files)} gefunden):")
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    print(f"\n📋 {csv_file.name}:")
    print(f"  Dimensionen: {df.shape[0]} Zeilen × {df.shape[1]} Spalten")
    print(f"  Spalten: {list(df.columns)}")

    # Erste Datenqualitätsindikatoren
    null_count = df.isnull().sum().sum()
    duplicate_count = df.duplicated().sum()
    print(f"  Qualität: {null_count} NULL-Werte, {duplicate_count} Duplikate")

🗂️ Bronze Tabellen (9 gefunden):

📋 CST_AZ12.csv:
  Dimensionen: 18484 Zeilen × 3 Spalten
  Spalten: ['CID', 'BDATE', 'GEN']
  Qualität: 1472 NULL-Werte, 0 Duplikate

📋 cst_info.csv:
  Dimensionen: 18494 Zeilen × 7 Spalten
  Spalten: ['cst_id', 'cst_key', 'cst_firstname', 'cst_lastname', 'cst_marital_status', 'cst_gndr', 'cst_create_date']
  Qualität: 4608 NULL-Werte, 0 Duplikate

📋 customer_info.csv:
  Dimensionen: 5 Zeilen × 5 Spalten
  Spalten: ['customer_id', 'firstname', 'lastname', 'gender', 'date_of_birth']
  Qualität: 0 NULL-Werte, 0 Duplikate

📋 LOC_A101.csv:
  Dimensionen: 18484 Zeilen × 2 Spalten
  Spalten: ['CID', 'CNTRY']
  Qualität: 332 NULL-Werte, 0 Duplikate

📋 prd_info.csv:
  Dimensionen: 397 Zeilen × 7 Spalten
  Spalten: ['prd_id', 'prd_key', 'prd_nm', 'prd_cost', 'prd_line', 'prd_start_dt', 'prd_end_dt']
  Qualität: 216 NULL-Werte, 0 Duplikate

📋 product_info.csv:
  Dimensionen: 4 Zeilen × 4 Spalten
  Spalten: ['product_id', 'product_name', 'category', 'price']
  Qua

## 🤖 Silver Draft Agent - Datenprofilierung

**Was passiert hier:**
- Simulation der automatischen Datenprofilierung des Silver Draft Agents
- Erkennung von Datenqualitätsproblemen (NULL-Werte, Duplikate, Formatierungsfehler)
- Generierung von Transformationsempfehlungen

**Agent-Logik:**
Der Silver Draft Agent analysiert jede Spalte auf:
- Datentyp-Konsistenz
- Whitespace-Probleme
- Leere Strings vs. NULL-Werte
- Duplikate auf Zeilenebene

In [4]:
def simulate_silver_draft_agent(df, filename):
    """Simuliert die Datenprofilierung des Silver Draft Agents"""
    profile = {
        "table": filename,
        "row_count": len(df),
        "column_count": len(df.columns),
        "columns": list(df.columns)
    }

    # Agent-Analyse: Identifiziere Transformationsbedarfe
    transforms = []
    quality_issues = []

    for col in df.columns:
        # Prüfe auf Whitespace-Probleme
        if df[col].dtype == 'object':
            if df[col].astype(str).str.strip().ne(df[col].astype(str)).any():
                transforms.append(f"🧹 Whitespace entfernen in '{col}'")
                quality_issues.append(f"Whitespace in {col}")

            # Prüfe auf leere Strings
            if (df[col] == "").any():
                transforms.append(f"🔄 Leere Strings zu NULL konvertieren in '{col}'")
                quality_issues.append(f"Leere Strings in {col}")

        # Prüfe auf NULL-Werte
        null_count = df[col].isnull().sum()
        if null_count > 0:
            transforms.append(f"⚠️ {null_count} NULL-Werte behandeln in '{col}'")
            quality_issues.append(f"{null_count} NULLs in {col}")

    # Prüfe auf Duplikate
    duplicate_count = df.duplicated().sum()
    if duplicate_count > 0:
        transforms.append(f"🗑️ {duplicate_count} doppelte Zeilen entfernen")
        quality_issues.append(f"{duplicate_count} doppelte Zeilen")

    profile["quality_issues"] = quality_issues
    profile["suggested_transforms"] = transforms if transforms else ["✅ Keine Transformationen erforderlich"]

    return profile

# Führe Agent-Simulation für alle Tabellen aus
print("🤖 Silver Draft Agent Analyse:")
table_profiles = {}

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    profile = simulate_silver_draft_agent(df, csv_file.name)
    table_profiles[csv_file.name] = profile

    print(f"\n📊 Analyse: {csv_file.name}")
    print(f"  Qualitätsprobleme: {len(profile['quality_issues'])}")
    for issue in profile['quality_issues']:
        print(f"    - {issue}")

    print(f"  Transformationsplan:")
    for transform in profile['suggested_transforms']:
        print(f"    - {transform}")

🤖 Silver Draft Agent Analyse:

📊 Analyse: CST_AZ12.csv
  Qualitätsprobleme: 2
    - Whitespace in GEN
    - 1472 NULLs in GEN
  Transformationsplan:
    - 🧹 Whitespace entfernen in 'GEN'
    - ⚠️ 1472 NULL-Werte behandeln in 'GEN'

📊 Analyse: cst_info.csv
  Qualitätsprobleme: 8
    - 4 NULLs in cst_id
    - Whitespace in cst_firstname
    - 8 NULLs in cst_firstname
    - Whitespace in cst_lastname
    - 7 NULLs in cst_lastname
    - 7 NULLs in cst_marital_status
    - 4578 NULLs in cst_gndr
    - 4 NULLs in cst_create_date
  Transformationsplan:
    - ⚠️ 4 NULL-Werte behandeln in 'cst_id'
    - 🧹 Whitespace entfernen in 'cst_firstname'
    - ⚠️ 8 NULL-Werte behandeln in 'cst_firstname'
    - 🧹 Whitespace entfernen in 'cst_lastname'
    - ⚠️ 7 NULL-Werte behandeln in 'cst_lastname'
    - ⚠️ 7 NULL-Werte behandeln in 'cst_marital_status'
    - ⚠️ 4578 NULL-Werte behandeln in 'cst_gndr'
    - ⚠️ 4 NULL-Werte behandeln in 'cst_create_date'

📊 Analyse: customer_info.csv
  Qualitätsprobleme:

## 📄 Silver Agent Context - LLM Output

**Was passiert hier:**
- Inspektion des tatsächlichen Outputs des Silver Draft Agents
- Überprüfung der vom LLM generierten Analyse und Empfehlungen
- Vergleich zwischen simulierter und echter Agent-Analyse

**Agent-Output:**
Der Silver Draft Agent erstellt strukturierte JSON-Kontextdaten, die der Silver Builder Agent verwendet, um ausführbaren Code zu generieren.

In [5]:
# Silver Agent Context analysieren
silver_context_path = silver_dir / "reports" / "silver_run_agent_context.json"
if silver_context_path.exists():
    with open(silver_context_path, 'r') as f:
        silver_context = json.load(f)

    print("🤖 Silver Draft Agent - LLM Output:")
    print(f"Run ID: {silver_context.get('run_id')}")
    print(f"Layer: {silver_context.get('layer')}")
    print(f"Quell-Layer: {silver_context.get('source_layer')}")

    # Performance Metriken
    perf = silver_context.get('performance_metrics', {})
    print(f"\n📈 Agent Performance:")
    print(f"  Zeilen verarbeitet: {perf.get('total_rows_processed', 0):,}")
    print(f"  Spalten verarbeitet: {perf.get('total_columns_processed', 0):,}")
    print(f"  Tabellen verarbeitet: {perf.get('tables_processed', 0)}")
    print(f"  Tabellen fehlgeschlagen: {perf.get('tables_failed', 0)}")

    # Schema-Übersicht vom Agent
    if 'profile' in silver_context:
        print(f"\n🔍 Agent Schema-Analyse:")
        schema = silver_context['profile'].get('schema_overview', {})
        for table in schema.get('tables', []):
            print(f"  📋 {table['table']}: {table['row_count']} Zeilen, {table['column_count']} Spalten")
else:
    print("❌ Silver Agent Context nicht gefunden - Agent wurde nicht ausgeführt")

🤖 Silver Draft Agent - LLM Output:
Run ID: 20260124_231016_#6470e523
Layer: silver
Quell-Layer: bronze

📈 Agent Performance:
  Zeilen verarbeitet: 116,311
  Spalten verarbeitet: 47
  Tabellen verarbeitet: 9
  Tabellen fehlgeschlagen: 0

🔍 Agent Schema-Analyse:
  📋 CST_AZ12.csv: 18484 Zeilen, 3 Spalten
  📋 cst_info.csv: 18494 Zeilen, 7 Spalten
  📋 customer_info.csv: 5 Zeilen, 5 Spalten
  📋 LOC_A101.csv: 18484 Zeilen, 2 Spalten
  📋 prd_info.csv: 397 Zeilen, 7 Spalten
  📋 product_info.csv: 4 Zeilen, 4 Spalten
  📋 PX_CAT_G1V2.csv: 37 Zeilen, 4 Spalten
  📋 sales_details.csv: 60398 Zeilen, 9 Spalten
  📋 sales_transactions.csv: 8 Zeilen, 6 Spalten


## 📝 Silver Human Report - LLM Insights

**Was passiert hier:**
- Anzeige des menschenlesbaren Reports des Silver Draft Agents
- Überprüfung der LLM-generierten Datenqualitätsanalyse
- Einsicht in die Agent-Empfehlungen für Business-Stakeholder

**Report-Inhalt:**
Der Agent erstellt einen strukturierten Markdown-Report mit Executive Summary, Datenqualitätsbewertung und Transformationsempfehlungen.

In [6]:
# Silver Human Report inspizieren
silver_report_path = silver_dir / "reports" / "silver_run_human_report.md"
if silver_report_path.exists():
    with open(silver_report_path, 'r', encoding='utf-8') as f:
        silver_report = f.read()

    print("📄 Silver Draft Agent - Human Report:")
    print("=" * 60)

    # Zeige ersten Teil des Reports
    lines = silver_report.split('\n')
    for i, line in enumerate(lines[:30]):  # Erste 30 Zeilen
        print(line)

    if len(lines) > 30:
        print(f"\n... ({len(lines)-30} weitere Zeilen)")

    print(f"\n📊 Report Statistiken:")
    print(f"  Gesamtlänge: {len(silver_report):,} Zeichen")
    print(f"  Zeilen: {len(lines)}")
    print(f"  Abschnitte: {silver_report.count('#')} (geschätzt)")
else:
    print("❌ Silver Human Report nicht gefunden")

📄 Silver Draft Agent - Human Report:
# Silver Layer Run Report  
**Run ID:** 20260124_231151_#6470e523  
**Bronze Run ID:** 20260124_231016_#6470e523  
**Run Timestamp:** 2026-01-24T23:10:17 UTC (Bronze layer)

---

## Executive Summary

- All nine source tables from CRM and ERP systems were successfully ingested in the Bronze layer with no failures detected.
- Silver layer transformations are planned to standardize missing values and data types without aggregations, ensuring clean, granular data ready for analytics.
- The data schema is structurally sound with no duplicate rows, well-defined keys, and mostly non-null primary identifiers.
- The Silver layer is well-prepared to support business problem exploration including customer retention, product and category performance, sales trends, and location-based analysis.
- Opportunities exist to define KPIs for BI/dashboarding and to apply machine learning methods for customer and product segmentation leveraging the cleaned data.

---

##

## 🥈 Silver Layer Ergebnisse

**Was passiert hier:**
- Inspektion der transformierten Silver-Daten
- Vergleich zwischen Bronze- und Silver-Versionen der Tabellen
- Überprüfung der Datenqualitätsverbesserungen

**Transformationsergebnisse:**
Der Silver Runner hat die vom Silver Builder Agent generierten Transformationen ausgeführt und bereinigte Daten erstellt.

In [13]:
from pathlib import Path

print("Reports-Inhalt:")
for p in sorted((silver_dir / "reports").rglob("*")):
    print(" -", p.relative_to(silver_dir))
    
print("\nSuche nach typischen Daten-Dateien unter silver_dir:")
patterns = ["*.csv", "*.parquet", "*.jsonl", "*.feather", "*.pkl"]
found = []
for pat in patterns:
    found += list(silver_dir.rglob(pat))

# metadata.yaml/report-Dateien rausfiltern
found = [p for p in found if p.name != "metadata.yaml" and "reports" not in p.parts]

print("Gefunden:", len(found))
for p in found[:30]:
    print(" -", p.relative_to(silver_dir))


Reports-Inhalt:
 - reports\elt_report.html
 - reports\silver_run_agent_context.json
 - reports\silver_run_human_report.md

Suche nach typischen Daten-Dateien unter silver_dir:
Gefunden: 0


In [10]:
# Silver Transformationsergebnisse analysieren
silver_data_dir = silver_dir / "data"
if silver_data_dir.exists():
    silver_csv_files = list(silver_data_dir.glob("*.csv"))

    print(f"🥈 Silver Layer Ergebnisse ({len(silver_csv_files)} Tabellen):")

    for csv_file in silver_csv_files:
        df_silver = pd.read_csv(csv_file)
        print(f"\n📊 {csv_file.name}:")
        print(f"  Silver: {df_silver.shape[0]} Zeilen × {df_silver.shape[1]} Spalten")

        # Vergleiche mit Bronze-Version
        bronze_equivalent = bronze_data_dir / csv_file.name
        if bronze_equivalent.exists():
            df_bronze = pd.read_csv(bronze_equivalent)

            # Datenqualitätsverbesserungen
            bronze_nulls = df_bronze.isnull().sum().sum()
            silver_nulls = df_silver.isnull().sum().sum()
            bronze_dupes = df_bronze.duplicated().sum()
            silver_dupes = df_silver.duplicated().sum()

            print(f"  📈 Qualitätsverbesserungen:")
            print(f"    Zeilen: {df_bronze.shape[0]} → {df_silver.shape[0]} ({df_silver.shape[0] - df_bronze.shape[0]:+d})")
            print(f"    NULL-Werte: {bronze_nulls} → {silver_nulls} ({silver_nulls - bronze_nulls:+d})")
            print(f"    Duplikate: {bronze_dupes} → {silver_dupes} ({silver_dupes - bronze_dupes:+d})")

            # Datentyp-Verbesserungen
            print(f"  🔄 Datentyp-Transformationen:")
            for col in df_silver.columns:
                if col in df_bronze.columns:
                    bronze_type = str(df_bronze[col].dtype)
                    silver_type = str(df_silver[col].dtype)
                    if bronze_type != silver_type:
                        print(f"    {col}: {bronze_type} → {silver_type}")
        else:
            print(f"  ⚠️ Keine Bronze-Vergleichsdaten gefunden")
else:
    print("❌ Silver Daten nicht gefunden - Transformation nicht ausgeführt")

❌ Silver Daten nicht gefunden - Transformation nicht ausgeführt


## 📊 Datenqualitäts-Dashboard

**Was passiert hier:**
- Erstellung einer Übersicht über die Datenqualitätsverbesserungen
- Quantifizierung der Agent-Performance
- Bewertung der Transformationserfolge

**Qualitätsmetriken:**
Vergleich der Datenqualität vor und nach der Silver-Transformation.

In [7]:
# Datenqualitäts-Dashboard erstellen
if silver_data_dir.exists() and bronze_data_dir.exists():
    print("📊 DATENQUALITÄTS-DASHBOARD")
    print("=" * 50)

    total_bronze_rows = 0
    total_silver_rows = 0
    total_bronze_nulls = 0
    total_silver_nulls = 0
    total_bronze_dupes = 0
    total_silver_dupes = 0
    tables_processed = 0

    for csv_file in silver_csv_files:
        bronze_file = bronze_data_dir / csv_file.name
        if bronze_file.exists():
            df_bronze = pd.read_csv(bronze_file)
            df_silver = pd.read_csv(csv_file)

            total_bronze_rows += len(df_bronze)
            total_silver_rows += len(df_silver)
            total_bronze_nulls += df_bronze.isnull().sum().sum()
            total_silver_nulls += df_silver.isnull().sum().sum()
            total_bronze_dupes += df_bronze.duplicated().sum()
            total_silver_dupes += df_silver.duplicated().sum()
            tables_processed += 1

    print(f"📈 GESAMTSTATISTIKEN:")
    print(f"  Tabellen verarbeitet: {tables_processed}")
    print(f"  Zeilen: {total_bronze_rows:,} → {total_silver_rows:,} ({total_silver_rows - total_bronze_rows:+,})")
    print(f"  NULL-Werte: {total_bronze_nulls:,} → {total_silver_nulls:,} ({total_silver_nulls - total_bronze_nulls:+,})")
    print(f"  Duplikate: {total_bronze_dupes:,} → {total_silver_dupes:,} ({total_silver_dupes - total_bronze_dupes:+,})")

    # Qualitätsscore berechnen
    if total_bronze_rows > 0:
        bronze_quality = (1 - (total_bronze_nulls + total_bronze_dupes) / (total_bronze_rows * 10)) * 100
        silver_quality = (1 - (total_silver_nulls + total_silver_dupes) / (total_silver_rows * 10)) * 100
        quality_improvement = silver_quality - bronze_quality

        print(f"\n🎯 QUALITÄTSSCORE:")
        print(f"  Bronze Layer: {bronze_quality:.1f}%")
        print(f"  Silver Layer: {silver_quality:.1f}%")
        print(f"  Verbesserung: {quality_improvement:+.1f}%")
else:
    print("❌ Kann Datenqualitäts-Dashboard nicht erstellen - Daten fehlen")

NameError: name 'silver_data_dir' is not defined

## 🔄 Agent-Workflow Zusammenfassung

**Was Sie gelernt haben:**
Dieser Abschnitt fasst den gesamten Silver Agent Workflow zusammen und erklärt die Rolle jedes Agents im Datenbereinigungsprozess.

In [ ]:
print("🤖 SILVER AGENT WORKFLOW - ZUSAMMENFASSUNG")
print("=" * 60)
print()
print("1️⃣ SILVER DRAFT AGENT (Analyse & Planung):")
print("   🔍 Liest Bronze-Metadaten und CSV-Dateien")
print("   📊 Führt automatische Datenprofilierung durch")
print("   ⚠️ Identifiziert Datenqualitätsprobleme")
print("   💡 Schlägt spezifische Transformationen vor")
print("   📄 Erstellt silver_run_agent_context.json")
print("   📝 Erstellt silver_run_human_report.md")
print()
print("2️⃣ SILVER BUILDER AGENT (Code-Generierung):")
print("   📖 Liest Silver Draft Context")
print("   🛠️ Generiert ausführbaren Python-Code")
print("   🔧 Implementiert vorgeschlagene Transformationen")
print("   📜 Erstellt load_2_silver_layer_runner.py")
print()
print("3️⃣ SILVER RUNNER (Ausführung):")
print("   ▶️ Führt generierten Code aus")
print("   🔄 Transformiert Bronze → Silver")
print("   💾 Erstellt bereinigte CSV-Dateien")
print("   📋 Generiert Metadaten und Reports")
print()
print("✅ ERGEBNIS: Bereinigte, standardisierte Silver-Layer Daten")
print("   - Konsistente Datentypen")
print("   - Entfernte Duplikate")
print("   - Standardisierte NULL-Werte")
print("   - Bereinigte Formatierung")
print("   - Vollständige Dokumentation")